 Prototype of the Machine Learning Pipeline

Load the necessary packages

In [ ]:
import sys
import os
import torch
import pickle
from speckcn2 import *

Load the configuration file<br>
this file contains all the information related to how to preprocess the data and which model to use

In [ ]:
config = load_config(sys.argv[1])
# one of which is the path to the data, that we need to store in this variable
datadirectory = config['speckle']['datadirectory']

Set the device: if a GPU is available, we use it

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device}.', flush=True)

Preprocess the data. If a database of the preprocess data exist, then we just load it

In [ ]:
all_images, all_tags, all_ensemble_ids = prepare_data(config, nimg_print=15)

Normalize the tags between 0 and 1. This will help the model by giving it more reasonable numbers to work with

In [ ]:
nz = Normalizer(config)

Split the data in training and testing

In [ ]:
train_set, test_set = train_test_split(all_images, all_tags, all_ensemble_ids, nz)

Load the model that you want to use

In [ ]:
model, last_model_state = setup_model(config)
# and set the model to run on the device
model = model.to(device)

Define the loss function and optimizer

In [ ]:
criterion = ComposableLoss(config, nz)
optimizer = setup_optimizer(config, model)

(!) Train the model

In [ ]:
model, average_loss = train(model, last_model_state, config, train_set,
                            test_set, device, optimizer, criterion)
print(f'Finished Training, Loss: {average_loss:.5f}', flush=True)

Now test the model, while also producing some plots

In [ ]:
test_tags = score(model, test_set, device, criterion, nz)

Finaly we do some postprocessing analysis<br>
Plot the distribution of the screen tags

In [ ]:
tags_distribution(config, train_set, test_tags, device)
tags_distribution(config, train_set, test_tags, device, rescale=True, recover_tag=nz.recover_tag)
# Plot the loss during training
plot_loss(config, model, datadirectory)
# Plot the execution time
plot_time(config, model, datadirectory)